In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
df_1 = pd.read_json("experiment_2_run_1.json")
df_2 = pd.read_json("experiment_2_run_2.json")
df_3 = pd.read_json("experiment_2_run_3.json")
df_1["passed"] = (df_1["status"] == "passed").astype("int")
df_2["passed"] = (df_2["status"] == "passed").astype("int")
df_3["passed"] = (df_3["status"] == "passed").astype("int")

In [3]:
df = df_1.copy()
df = df.drop(columns=["_id", "__v", "model", "instructions_prompt", "collection", "answer", "log", "messages", "number_of_response_messages", "average_message_length", "comment"])
df["passed"] = df_1["passed"] + df_2["passed"] + df_3["passed"]

In [4]:
df.head()

,name,visualization,question_item,task,visual,question,dataset,status,passed
0,Bubble Chart test item 1 gpt-4o prompt vundefined,Bubble Chart,1,Retrieve Value,no,What is the total length of the trail lines in...,BubbleChart.csv,passed,3
1,Bubble Chart test item 2 gpt-4o prompt vundefined,Bubble Chart,2,Retrieve Value,no,What is the number of stations in Shanghai?,BubbleChart.csv,passed,3
2,Bubble Chart test item 3 gpt-4o prompt vundefined,Bubble Chart,3,Retrieve Value,yes,What is the y-value of the point corresponding...,BubbleChart.csv,passed,3
3,Bubble Chart test item 4 gpt-4o prompt vundefined,Bubble Chart,4,Retrieve Value,yes,What is the x-value of the point corresponding...,BubbleChart.csv,passed,3
4,Bubble Chart test item 5 gpt-4o prompt vundefined,Bubble Chart,5,Filter,no,How many cities have more than 700 stations?,BubbleChart.csv,passed,3


In [5]:
df["passed"].value_counts()

passed
3    99
1     3
0     3
2     2
Name: count, dtype: int64

# Aggregate Runs
## Which questions are answered wrong in each run?

In [6]:
df[df["passed"] == 0]#[["question_item", "visualization", "task", "model", "prompt", "question"]]

,name,visualization,question_item,task,visual,question,dataset,status,passed
34,Candlestick Chart test item 11 gpt-4o prompt v...,Candlestick Chart,11,Compute Derived Value,yes,What box has the largest width?,Volkswagen_Candlestick.csv,failed,0
35,Candlestick Chart test item 12 gpt-4o prompt v...,Candlestick Chart,12,Compute Derived Value,yes,"Which box, i.e., which number, has the largest...",Volkswagen_Candlestick.csv,failed,0
105,Scatterplot test item 26 gpt-4o prompt vundefined,Scatterplot,26,Find Anomalies,yes,Are there points that seem to be outliers?,scatter.csv,failed,0


## Which questions are answered inconsistently? (sometimes right, sometimes wrong)

In [7]:
df[df["passed"].isin([1,2])]

,name,visualization,question_item,task,visual,question,dataset,status,passed
24,Candlestick Chart test item 1 gpt-4o prompt vu...,Candlestick Chart,1,Retrieve Value,no,What was the opening price in Week 31?,Volkswagen_Candlestick.csv,passed,2
26,Candlestick Chart test item 3 gpt-4o prompt vu...,Candlestick Chart,3,Retrieve Value,yes,What is the upper bound of the 31st box?,Volkswagen_Candlestick.csv,failed,1
42,Candlestick Chart test item 19 gpt-4o prompt v...,Candlestick Chart,19,Sort,yes,Which three weeks have the green boxes with th...,Volkswagen_Candlestick.csv,failed,1
43,Candlestick Chart test item 20 gpt-4o prompt v...,Candlestick Chart,20,Sort,yes,Which three weeks have the red boxes with the ...,Volkswagen_Candlestick.csv,passed,2
46,Candlestick Chart test item 23 gpt-4o prompt v...,Candlestick Chart,23,Determine Range,yes,In which range do the boxes or their vertical ...,Volkswagen_Candlestick.csv,failed,1


## apply correct passed value

In [8]:
df["passed"] = np.where(df["passed"] == 3, 1, 0)
df.passed.value_counts()

passed
1    99
0     8
Name: count, dtype: int64

# Analysis
## By Vis

In [9]:
vis = df.groupby(["visualization", "passed"]).size().unstack(fill_value=0)
vis.columns.name = None
vis["hit_rate"] = vis[1] / (vis[1] + vis[0])
vis = vis.reset_index()

#vis.to_csv("df_vis_experiment_2.csv", index=False)
vis

,visualization,0,1,hit_rate
0,Bubble Chart,0,24,1.000000
1,Candlestick Chart,7,17,0.708333
2,Choropleth,0,11,1.000000
3,Line Chart,0,21,1.000000
4,Scatterplot,1,26,0.962963


## By Task

In [10]:
task = df.groupby(["task", "passed"]).size().unstack(fill_value=0)
task.columns.name = None
task["hit_rate"] = task[1] / (task[1] + task[0])
task = task.reset_index()

#task.to_csv("df_task_experiment_2.csv", index=False)
task

,task,0,1,hit_rate
0,Compute Derived Value,2,15,0.882353
1,Correlate,0,1,1.000000
2,Determine Range,1,14,0.933333
3,Filter,0,18,1.000000
4,Find Anomalies,1,1,0.500000
5,Find Extremum,0,18,1.000000
6,Retrieve Value,2,16,0.888889
7,Sort,2,16,0.888889
